<a href="https://colab.research.google.com/github/aashrithresearch/pytorch_pathology/blob/main/epistroma_hdf5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.6.1.


In [ ]:
dataname = 'epistroma' #will help for using paths
patch_size = 500 #sz of tiles to extract
stride_size = 250 #dst between patches
mirror_pad_size = 250 #num pixels to pad after resz
test_set_size = 0.1
resize = 1 #resz input images
classes = [0, 1] #classes labels w nums

In [ ]:
import torch
import tables

import os,sys
import glob

import PIL
import numpy as np

import cv2
import matplotlib.pyplot as plt
from sklearn import model_selection

import random
import numbers
from numpy.lib.stride_tricks import as_strided

seed = random.randrange(sys.maxsize)
random.seed(seed)
print(f"random seed: {seed}")

random seed: 8265515718574625133


In [ ]:
def extract_patches(arr, patch_shape=8, extraction_step=1):
    arr_ndim = arr.ndim

    if isinstance(patch_shape, numbers.Number):
        patch_shape = tuple([patch_shape] * arr_ndim)
    if isinstance(extraction_step, numbers.Number):
        extraction_step = tuple([extraction_step] * arr_ndim)

    patch_strides = arr.strides

    slices = tuple(slice(None, None, st) for st in extraction_step)
    indexing_strides = arr[slices].strides

    patch_indices_shape = (
        (np.array(arr.shape) - np.array(patch_shape)) // np.array(extraction_step)
    ) + 1

    shape = tuple(list(patch_indices_shape) + list(patch_shape))
    strides = tuple(list(indexing_strides) + list(patch_strides))

    patches = as_strided(arr, shape=shape, strides=strides)
    return patches

In [ ]:
img_dtype = tables.UInt8Atom()
filenameAtom = tables.StringAtom(itemsize=255)

In [ ]:
files=glob.glob('/content/drive/MyDrive/epi/masks/*.png') #files list

#training/valid phases
phases={}
phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))

#creating 2 images types
imgtypes=["img","mask"]

In [ ]:
storage={}

block_shape={}
block_shape["img"]= np.array((patch_size,patch_size,3))
block_shape["mask"]= np.array((patch_size,patch_size))

filters=tables.Filters(complevel=6, complib='zlib')


for phase in phases.keys():
    print(phase)

    totals=np.zeros((2,len(classes)))
    totals[0,:]=classes

    hdf5_file = tables.open_file(f"/content/drive/MyDrive/epi/{dataname}_{phase}.pytable", mode='w')
    storage["filename"] = hdf5_file.create_earray(hdf5_file.root, 'filename', filenameAtom, (0,))

    for imgtype in imgtypes:
        storage[imgtype]= hdf5_file.create_earray(hdf5_file.root, imgtype, img_dtype,
                                                  shape=np.append([0],block_shape[imgtype]),
                                                  chunkshape=np.append([1],block_shape[imgtype]),
                                                  filters=filters)

    for filei in phases[phase]:
        fname=files[filei]

        print(fname)
        for imgtype in imgtypes:
            print(imgtype)
            if(imgtype=="img"):
                io=cv2.cvtColor(cv2.imread("/content/drive/MyDrive/epi/"+os.path.basename(fname).replace("_mask.png",".tif")),cv2.COLOR_BGR2RGB)
                interp_method=PIL.Image.BICUBIC

            else:
                io=cv2.imread(fname)/255
                interp_method=PIL.Image.NEAREST

                for i,key in enumerate(classes):
                    totals[1,i]+=sum(sum(io[:,:,0]==key))


            io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method)
            io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

            io_arr_out=extract_patches(io,(patch_size,patch_size,3),stride_size)

            io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)



            if(imgtype=="img"):
                storage[imgtype].append(io_arr_out)
            else:
                storage[imgtype].append(io_arr_out[:,:,:,0].squeeze())

        storage["filename"].append([fname for x in range(io_arr_out.shape[0])])

    npixels=hdf5_file.create_carray(hdf5_file.root, 'numpixels', tables.Atom.from_dtype(totals.dtype), totals.shape)
    npixels[:]=totals
    hdf5_file.close()

train
/content/drive/MyDrive/epi/masks/9227_00006_mask.png
img
mask
/content/drive/MyDrive/epi/masks/12820_00005_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10302_00098_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10261_00002_mask.png
img
mask
/content/drive/MyDrive/epi/masks/9346_00019_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10282_00016_mask.png
img
mask
/content/drive/MyDrive/epi/masks/9023_00012_mask.png
img
mask
/content/drive/MyDrive/epi/masks/8957_00005_mask.png
img
mask
/content/drive/MyDrive/epi/masks/9043_00045_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10295_00012_mask.png
img
mask
/content/drive/MyDrive/epi/masks/12947_00004_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10286_00014_mask.png
img
mask
/content/drive/MyDrive/epi/masks/12819_00004_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10304_00005_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10291_00012_mask.png
img
mask
/content/drive/MyDrive/epi/masks/10285_00007_mask.png
